In [1]:
Liszt <- read.csv("homework.csv", header=TRUE, sep=",")

Same as for the other homeworks, we first define the dataset.

In [2]:
Liszt$TGiving=Liszt$FY04Giving+Liszt$FY03Giving+Liszt$FY02Giving+Liszt$FY01Giving+Liszt$FY00Giving

Here we calculate the total donation per student so we can later segment our dataset.

Since we are asked to "construct the 95% confidence interval for the mean of total contributions and interpret this interval" we first need to calculate the mean of total contributions.

In [3]:
meana=mean(Liszt$TGiving)

Now that we have the mean of total contributions, we need to calculate the z value. By following the code given in course #4, we can determine the correct z value for a 95% confidence interval. 0.95 corresponds to 90% confidence interval and 5% left in each tail therefore, if we want to know the number for 95% confidence interval we can have the following logic: if 0.95 is 5% in each tail, then to have 2.5% in each tail which corresponds to 95% CI then 0.975 is the correct number for z.

In [4]:
za=qnorm(0.975)

For our formula, we need to have the total number of contribution, it can be calculated by counting the number of rows in the dataset.

In [5]:
numbera <- nrow(Liszt)

So for the first CI we have the following calculation

In [6]:
CIza=meana+c(-1,1)*za*sd(Liszt$TGiving)/sqrt(numbera)

In [7]:
print(CIza)

[1]  607.2472 1352.8399


So with our result we can see that the mean of total contribution is between 607.2472 and 1352.8399. 

In [8]:
b=Liszt[Liszt$TGiving>=1000,]

2. Estimate the proportion of alumni who donate not less than 1000 in total with 95% confidence interval and interpret it.

To segment our dataset, we use the code given in the course.

In [9]:
numberb <- nrow(b)

So we need to count again the number of row which is 136 here.

In [10]:
p = numberb/numbera

The proportion here can be calculated by the number of row segmented divided by the total number of population in our data set.

In [11]:
zb=qnorm(0.975)

We use the same norm as the first question since our CI is also 95% for this question. We then use the code given in course #4 calculation for "confidence interval generated for proportion". 

In [12]:
CIzb=p+c(-1,1)*zb*sqrt(p*(1-p)/numbera)

In [13]:
print(CIzb)

[1] 0.09304368 0.12809453


Here the proportion for CI is between 9.30% and 12.81%. It means that in our data set, there are between 9.30% and 12.81% of people who donate an amount equal or exceeding $1000.

For the final question where we need to determine the minimum sample size.

In [14]:
e=100

Here e is the "mean total contribution which is required to decrease to 100".

We then apply the formula to calculate the sample size.

In [15]:
N=(za*sd(Liszt$TGiving)/e)^2

In [16]:
print(N)

[1] 17094.19


Our required sample size is 17094 which is more than 10 times the original sample size (1230). Consequently, we can say that the current sample is insufficient for the parameters we want to use.